# テキスト前処理

In [1]:
with open("pdf_to_text/pdf_to_text_0.txt",mode="r",encoding="utf-8") as f:
    text_data = f.read()

In [2]:
text_data[:1000]

'ビジネス光ネットサービス契約約款 \n\n２０２３年８月１日 \n\n株式会社  ＳＴＮｅｔ \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\x0c\n目    次 \n\n第１章  総  則 \n\n第  １条  約款の適用・・・・・・・・・・・・・・・・・・・・・・・・・・１ \n第  ２条  約款の変更・・・・・・・・・・・・・・・・・・・・・・・・・・１ \n第  ３条  用語の定義・・・・・・・・・・・・・・・・・・・・・・・・・・１ \n\n第２章  光ネットサービスの種類等 \n\n第  ４条  光ネットサービスの種類等・・・・・・・・・・・・・・・・・・・３ \n\n第３章  光ネットサービスの提供区域等 \n\n第  ５条  光ネットサービスの提供区域・・・・・・・・・・・・・・・・・・３ \n第  ６条  光ネットサービスの提供範囲等・・・・・・・・・・・・・・・・・３ \n\n第４章  契  約 \n\n第  ７条  契約の単位・・・・・・・・・・・・・・・・・・・・・・・・・・３ \n第  ８条  契約者回線の終端・・・・・・・・・・・・・・・・・・・・・・・３ \n第  ９条  収容光ネットサービス取扱局・・・・・・・・・・・・・・・・・・３ \n第１０条  光ネット申込みの方法・・・・・・・・・・・・・・・・・・・・・４ \n第１１条  光ネット申込みの承諾・・・・・・・・・・・・・・・・・・・・・４ \n第１２条  提供開始日・・・・・・・・・・・・・・・・・・・・・・・・・・４ \n第１３条  最低利用期間・・・・・・・・・・・・・・・・・・・・・・・・・４ \n第１４条  品目及び種別等の変更・・・・・・・・・・・・・・・・・・・・・５ \n第１５条  契約者回線等の移転・・・・・・・・・・・・・・・・・・・・・・５ \n第１６条  その他の契約内容の変更・・・・・・・・・・・・・・・・・・・・５ \n第１７条  利用の一時中断・・・・・・・・・・・・・・・・・・・・・・・・５ \n第１８条  光ネットサービス契約に基づく債権・債務の譲渡の禁止・・・・・・５ \n第１９条  光ネットサービス契約者が行う光ネットサービス契約の解除・・・・５ \n第２０条  当社が